# Create Driver Summary (Preferences)
- This notebook take about 5 to 10 minutes to execute
- We will create 2 summaries
  - The driver preferences based upon what themes they mention in their reviews
  - A summary of all their reviews for a consolidated overall review

## Create Summary Prompt and run through LLM

In [ ]:
%%bigquery

-- The table is populated by: CALL ${project_id}.${bigquery_rideshare_llm_enriched_dataset}.sp_step_02_extract_driver_attributes()
SELECT *
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute`
ORDER BY driver_id,
         driver_attribute_grouping,
         extracted_driver_attribute,
         rank_order
LIMIT 100;

In [ ]:
%%bigquery

-- Distinct list of driver attributes

SELECT DISTINCT extracted_driver_attribute
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute`

In [ ]:
%%bigquery

-- Insert safe driver if they do not have any "unsafe attributed"
INSERT INTO `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute`
(driver_id, extracted_driver_attribute, driver_attribute_grouping, cnt, rank_order)
SELECT driver_id, 'safe driver', 'safe-driver', 1, 1
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
  WHERE driver.include_in_llm_processing = TRUE
    AND EXISTS     (SELECT *
                     FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute` AS driver_attribute
                    WHERE driver.driver_id = driver_attribute.driver_id)
    AND NOT EXISTS (SELECT *
                     FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute` AS driver_attribute
                    WHERE driver.driver_id = driver_attribute.driver_id
                      AND TRIM(extracted_driver_attribute) = 'distracted driver')
    AND NOT EXISTS (SELECT *
                     FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute` AS driver_attribute
                    WHERE driver.driver_id = driver_attribute.driver_id
                      AND TRIM(extracted_driver_attribute) = 'driving too fast')
    AND NOT EXISTS (SELECT *
                     FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute` AS child
                    WHERE driver.driver_id = child.driver_id
                      AND child.extracted_driver_attribute = 'safe driver')

In [ ]:
%%bigquery

-- Sample prompt aggregation of attributes
SELECT driver_id,
       STRING_AGG(
       CASE WHEN TRIM(extracted_driver_attribute) = 'driver likes music'           THEN 'Attribute: the driver likes the radio on\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes no music'        THEN 'Attribute: the driver likes the radio off\n'
            WHEN TRIM(extracted_driver_attribute) = 'trunk space large'            THEN 'Attribute: the driver has a large amount of trunk space\n'
            WHEN TRIM(extracted_driver_attribute) = 'trunk space small'            THEN 'Attribute: the driver has a small trunk\n'
            WHEN TRIM(extracted_driver_attribute) = 'Category: trunk space small'  THEN 'Attribute: the driver has a small trunk\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes conversation'    THEN 'Attribute: the driver likes to have a conversation with the passengers\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes no conversation' THEN 'Attribute: the driver does not like to talk to the passengers\n'
            WHEN TRIM(extracted_driver_attribute) = 'driving too fast'             THEN 'Attribute: the driver tends to drive too fast\n'
            WHEN TRIM(extracted_driver_attribute) = 'driving too slow'             THEN 'Attribute: the driver drives too slow\n'
            WHEN TRIM(extracted_driver_attribute) = 'clean car'                    THEN 'Attribute: the driver keeps their car clean on the inside and out\n'
            WHEN TRIM(extracted_driver_attribute) = 'dirty car'                    THEN 'Attribute: the driver has a dirty car\n'
            WHEN TRIM(extracted_driver_attribute) = 'car too hot'                  THEN 'Attribute: the driver has a keeps their car too warm inside\n'
            WHEN TRIM(extracted_driver_attribute) = 'car too cold'                 THEN 'Attribute: the driver keeps their car too cold\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver speaks spanish'        THEN 'Attribute: the driver is bilingual\n'
            WHEN TRIM(extracted_driver_attribute) = 'distracted driver'            THEN 'Attribute: the driver is a distracted driver\n'
            WHEN TRIM(extracted_driver_attribute) = 'safe driver'                  THEN 'Attribute: the driver is a safe driver\n'
            ELSE ''
       END,'') AS driver_attribute_agg
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute`
GROUP BY driver_id
LIMIT 100;

In [ ]:
%%bigquery

-- OPTIONAL: Reset all the fields to null
-- If you need to reset you data back to fresh data run the stored procedure: CALL `${project_id}.${bigquery_rideshare_llm_curated_dataset}.sp_reset_demo`();

/*
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
   SET driver_attribute_llm_summary_prompt = NULL,
       llm_summary_driver_attribute_json   = NULL,
       llm_summary_driver_attribute        = NULL
 WHERE TRUE;
*/

In [ ]:
%%bigquery

-- Create the LLM prompt
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
   SET driver_attribute_llm_summary_prompt =
       CONCAT('Write a 100 to 500 word summary for the following attributes.\n',
              '1. Randomly sort the attributes\n',
              '2. Select the first 3 to 5 attributes\n',
              '3. Write the summary in present tense for only the first 3 to 5 attributes\n',
              '4. The driver\'s name is ', driver.driver_name, '\n',
              '5. Use sentences with varying lengths\n',
              '6. Write 2 to 5 sentences',
               driver_attribute_agg),
       llm_summary_driver_attribute_json = NULL,
       llm_summary_driver_attribute      = NULL
  FROM
  (SELECT driver_id,
       STRING_AGG(
       CASE WHEN TRIM(extracted_driver_attribute) = 'driver likes music'           THEN 'Attribute: the driver likes the radio on\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes no music'        THEN 'Attribute: the driver likes the radio off\n'
            WHEN TRIM(extracted_driver_attribute) = 'trunk space large'            THEN 'Attribute: the driver has a large amount of trunk space\n'
            WHEN TRIM(extracted_driver_attribute) = 'trunk space small'            THEN 'Attribute: the driver has a small trunk\n'
            WHEN TRIM(extracted_driver_attribute) = 'Category: trunk space small'  THEN 'Attribute: the driver has a small trunk\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes conversation'    THEN 'Attribute: the driver likes to have a conversation with the passengers\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes no conversation' THEN 'Attribute: the driver does not like to talk to the passengers\n'
            WHEN TRIM(extracted_driver_attribute) = 'driving too fast'             THEN 'Attribute: the driver tends to drive too fast\n'
            WHEN TRIM(extracted_driver_attribute) = 'driving too slow'             THEN 'Attribute: the driver drives too slow\n'
            WHEN TRIM(extracted_driver_attribute) = 'clean car'                    THEN 'Attribute: the driver keeps their car clean on the inside and out\n'
            WHEN TRIM(extracted_driver_attribute) = 'dirty car'                    THEN 'Attribute: the driver has a dirty car\n'
            WHEN TRIM(extracted_driver_attribute) = 'car too hot'                  THEN 'Attribute: the driver has a keeps their car too warm inside\n'
            WHEN TRIM(extracted_driver_attribute) = 'car too cold'                 THEN 'Attribute: the driver keeps their car too cold\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver speaks spanish'        THEN 'Attribute: the driver is bilingual\n'
            WHEN TRIM(extracted_driver_attribute) = 'distracted driver'            THEN 'Attribute: the driver is a distracted driver\n'
            WHEN TRIM(extracted_driver_attribute) = 'safe driver'                  THEN 'Attribute: the driver is a safe driver\n'
            ELSE ''
       END,'') AS driver_attribute_agg
   FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute`
  GROUP BY driver_id
   ) AS driver_attribute
WHERE driver.driver_id = driver_attribute.driver_id;


In [ ]:
%%bigquery
SELECT driver_attribute_llm_summary_prompt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
 WHERE driver_attribute_llm_summary_prompt IS NOT NULL
 LIMIT 10;

## Run the LLM to generate a Driver Summary on Discovered Attributes

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

In [ ]:
# Process in batches
batch_size = 100

# Set the parameters so we are more deterministic and less creative/random responses
llm_temperature = 1
llm_max_output_tokens = 1024
llm_top_p = 1
llm_top_k = 40

update_sql="""
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
  SET llm_summary_driver_attribute_json = child.ml_generate_text_result
  FROM (SELECT *
          FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_enriched_dataset}.cloud_ai_llm_v1`,
              (SELECT driver_id,
                      driver_attribute_llm_summary_prompt AS prompt
                FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
                WHERE (llm_summary_driver_attribute_json IS NULL
                       OR
                       JSON_VALUE(llm_summary_driver_attribute_json, '$.predictions[0].content') IS NULL
                       )
                  AND include_in_llm_processing = TRUE
                  AND driver_attribute_llm_summary_prompt IS NOT NULL
                LIMIT {batch_size}),
              STRUCT(
                {llm_temperature} AS temperature,
                {llm_max_output_tokens} AS max_output_tokens,
                {llm_top_p} AS top_p,
                {llm_top_k} AS top_k
                ))
  ) AS child
WHERE driver.driver_id = child.driver_id
  """.format(batch_size = batch_size,
             llm_temperature = llm_temperature,
             llm_max_output_tokens = llm_max_output_tokens,
             llm_top_p = llm_top_p,
             llm_top_k = llm_top_k)

print("SQL: {update_sql}".format(update_sql=update_sql))


In [ ]:
# Score while records remain
# score in groups of batch_size records (we can do up to 10,000 at a time)
import time

done = False
displayed_first_sql = False
original_record_count = 0

while done == False:
  # Get the count of records to score
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
         WHERE (llm_summary_driver_attribute_json IS NULL
                OR
                JSON_VALUE(llm_summary_driver_attribute_json, '$.predictions[0].content') IS NULL
                )
           AND include_in_llm_processing = TRUE
           AND driver_attribute_llm_summary_prompt IS NOT NULL;
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)

  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))


## Parse the LLM JSON results

In [ ]:
%%bigquery

UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
   SET llm_summary_driver_attribute = JSON_VALUE(llm_summary_driver_attribute_json, '$.predictions[0].content')
 WHERE llm_summary_driver_attribute_json IS NOT NULL
   AND llm_summary_driver_attribute IS NULL;

In [ ]:
%%bigquery

SELECT driver_id, driver_attribute_llm_summary_prompt, llm_summary_driver_attribute
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
 WHERE llm_summary_driver_attribute_json IS NOT NULL
   AND llm_summary_driver_attribute IS NOT NULL
LIMIT 20;

# Create Driver Summary baesd upon Customer Reviews

## Create Summary Prompt and run through LLM

In [ ]:
%%bigquery

-- OPTIONAL: Reset all the fields to null
-- If you need to reset you data back to fresh data run the stored procedure: CALL `${project_id}.${bigquery_rideshare_llm_curated_dataset}.sp_reset_demo`();

/*
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
   SET driver_review_summary_llm_summary_prompt = NULL,
       llm_summary_driver_review_summary_json   = NULL,
       llm_summary_driver_review_summary        = NULL
 WHERE TRUE;
*/

In [ ]:
%%bigquery

-- Create the LLM prompt (latest 50 customer reviews per driver)
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
   SET driver_review_summary_llm_summary_prompt =
       CONCAT('Write a 100 to 600 word summary for the following customer reviews.\n',
              '1. Write the summary in present tense.\n',
              '2. Write the summary from the customers prespective.\n',
              '3. Do not repeat the same subject in the summary.\n',
              '4. The reviews are for the driver ', driver_name, ' at a rideshare company.\n',
              '5. The reviews are written by different customers.\n',
              '6. Write 3 to 6 sentences.\n',
               customer_review_agg)
  FROM (SELECT customer_review.driver_id,
               STRING_AGG(CONCAT("Review: ",customer_review_text),'\n') AS customer_review_agg
          FROM (SELECT driver_id,
                       customer_review_text
                  FROM (SELECT trip.driver_id,
                               trip.pickup_time,
                               customer_review.customer_review_text,
                               ROW_NUMBER() OVER (PARTITION BY trip.driver_id ORDER BY trip.pickup_time) AS row_nbr
                          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
                               INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.trip` AS trip
                                       ON customer_review.trip_id = trip.trip_id
                       ) AS reviews
                 WHERE row_nbr < 50 -- top 50 most recent
               ) AS customer_review
        GROUP BY driver_id) AS top_reviews
WHERE driver.driver_id = top_reviews.driver_id;


In [ ]:
%%bigquery

SELECT driver_review_summary_llm_summary_prompt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
WHERE driver_review_summary_llm_summary_prompt IS NOT NULL
LIMIT 5;


## Run the LLM to generate a driver summary of all customer reviews

In [ ]:
# Process in batches
batch_size = 10

# Set the parameters for more creative
llm_temperature = 1
llm_max_output_tokens = 1024
llm_top_p = 1
llm_top_k = 40

update_sql="""
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
  SET llm_summary_driver_review_summary_json = child.ml_generate_text_result
  FROM (SELECT *
          FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_enriched_dataset}.cloud_ai_llm_v1`,
              (SELECT driver_id,
                      driver_review_summary_llm_summary_prompt AS prompt
                FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
                WHERE (llm_summary_driver_review_summary_json IS NULL
                       OR
                       JSON_VALUE(llm_summary_driver_review_summary_json, '$.predictions[0].content') IS NULL
                       )
                  AND include_in_llm_processing = TRUE
                  AND driver_review_summary_llm_summary_prompt IS NOT NULL
                LIMIT {batch_size}),
              STRUCT(
                {llm_temperature} AS temperature,
                {llm_max_output_tokens} AS max_output_tokens,
                {llm_top_p} AS top_p,
                {llm_top_k} AS top_k
                ))
  ) AS child
WHERE driver.driver_id = child.driver_id
  """.format(batch_size = batch_size,
             llm_temperature = llm_temperature,
             llm_max_output_tokens = llm_max_output_tokens,
             llm_top_p = llm_top_p,
             llm_top_k = llm_top_k)

print("SQL: {update_sql}".format(update_sql=update_sql))


In [ ]:
# Score while records remain
# score in groups of batch_size records (we can do up to 10,000 at a time)
import time

done = False
displayed_first_sql = False
original_record_count = 0

while done == False:
  # Get the count of records to score
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
         WHERE (llm_summary_driver_review_summary_json IS NULL
                OR
                JSON_VALUE(llm_summary_driver_review_summary_json, '$.predictions[0].content') IS NULL
                )
           AND include_in_llm_processing = TRUE
           AND driver_review_summary_llm_summary_prompt IS NOT NULL;
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)


  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))


## Parse the LLM JSON results

In [ ]:
%%bigquery

UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` driver
   SET llm_summary_driver_review_summary = JSON_VALUE(llm_summary_driver_review_summary_json, '$.predictions[0].content')
 WHERE llm_summary_driver_review_summary_json IS NOT NULL
   AND llm_summary_driver_review_summary IS NULL;

In [ ]:
%%bigquery

SELECT driver_id, driver_review_summary_llm_summary_prompt, llm_summary_driver_review_summary_json, llm_summary_driver_review_summary
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
 WHERE llm_summary_driver_review_summary_json IS NOT NULL
   AND llm_summary_driver_review_summary IS NOT NULL
LIMIT 20;
